## 👥 Autores

Este proyecto fue desarrollado por:

| Nombre Completo                     | Número de Identificación | Carrera Universitaria              |
| ----------------------------------- | -----------------------: | ---------------------------------- |
| **Yorladys Argumedo Lozano**        | `1038824209`            | Ingeniería Industrial Virtual      |
| **Sebastian Gabriel Castro**        | `1029720632`            | Ingeniería Industrial Virtual      |

---


In [ ]:
# Importamos la librería de Google Colab para manejar archivos
from google.colab import files

# Instalamos la librería de Kaggle
!pip install kaggle --quiet

print("Por favor, sube el archivo 'kaggle.json' que descargaste de Kaggle:")
# Este comando abrirá una ventana para que selecciones el archivo desde tu PC
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# 1. Creamos la carpeta donde Kaggle espera encontrar las credenciales
!mkdir -p ~/.kaggle

# 2. Copiamos el archivo .json a esa carpeta
!cp kaggle.json ~/.kaggle/

# 3. Le damos los permisos de seguridad correctos al archivo
!chmod 600 ~/.kaggle/kaggle.json

print("¡API de Kaggle configurada!")

# 4. AHORA SÍ: Usamos la API para descargar los datos de la competencia
# Este comando le ordena a Colab que descargue los archivos por ti
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

print("\n¡Datos de la competencia descargados!")

# 5. Descomprimimos el archivo .zip que se acaba de descargar
!unzip -q -o udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

print("¡Archivos descomprimidos y listos para usar!")

¡API de Kaggle configurada!
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.31GB/s]

¡Datos de la competencia descargados!
¡Archivos descomprimidos y listos para usar!


In [ ]:
# ============================================================
# PREPROCESAMIENTO ROBUSTO - Versión mejorada
# ============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import os

# ------------------------------------------------------------
# 1. Cargar datos
# ------------------------------------------------------------
train_path = "train.csv"  # Ajusta la ruta si es necesario
train_df = pd.read_csv(train_path)

print(f"✅ DataFrame cargado correctamente: {train_df.shape}")
print(f"Columnas: {list(train_df.columns)}\n")

# ------------------------------------------------------------
# 2. Detectar columnas y preparar variable objetivo
# ------------------------------------------------------------
target_col = "RENDIMIENTO_GLOBAL"

if target_col not in train_df.columns:
    raise ValueError(f"La columna objetivo '{target_col}' no existe en el DataFrame.")

# Mapeo fijo para el target
target_mapping = {'alto': 0, 'bajo': 1, 'medio-alto': 2, 'medio-bajo': 3}
train_df[f"{target_col}_encoded_auto"] = train_df[target_col].map(target_mapping)

# Identificar tipos
id_cols = [col for col in train_df.columns if col.lower() in ["id", "identificador"]]
num_cols = train_df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in train_df.columns if c not in num_cols + id_cols + [target_col, f"{target_col}_encoded_auto"]]

# ------------------------------------------------------------
# 3. Limpiar texto y unificar categorías
# ------------------------------------------------------------
for c in cat_cols:
    train_df[c] = train_df[c].astype(str).str.strip().replace({"nan": np.nan, "None": np.nan})

# Mapear respuestas tipo Sí/No, Verdadero/Falso
bin_map = {"Si": 1, "Sí": 1, "SÍ": 1, "No": 0, "NO": 0, "si": 1, "no": 0, "True": 1, "False": 0}
for c in cat_cols:
    unique_vals = train_df[c].dropna().unique()
    if all(val in bin_map for val in unique_vals):
        train_df[c] = train_df[c].map(bin_map).astype("float")
        num_cols.append(c)
cat_cols = [c for c in cat_cols if c not in num_cols]

# ------------------------------------------------------------
# 4. Manejo de categorías con cardinalidad alta/baja
# ------------------------------------------------------------
cat_ohe = [c for c in cat_cols if train_df[c].nunique() <= 50]
cat_precode = [c for c in cat_cols if c not in cat_ohe]

# Precodificar las de alta cardinalidad
for c in cat_precode:
    train_df[c + "_code"] = pd.Categorical(train_df[c]).codes
    num_cols.append(c + "_code")

# ------------------------------------------------------------
# 5. Construir pipelines
# ------------------------------------------------------------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_ohe)
    ],
    remainder="drop"
)

# ------------------------------------------------------------
# 6. Entrenar y transformar (con chunks grandes)
# ------------------------------------------------------------
print(f"\n🧩 Columnas numéricas: {len(num_cols)} | Categóricas OHE: {len(cat_ohe)} | Precodificadas: {len(cat_precode)}")

X = train_df[num_cols + cat_ohe].copy()
y = train_df[f"{target_col}_encoded_auto"]

# Ajustar el pipeline con una muestra (para evitar sobrecarga)
sample_fit = X.sample(min(20000, len(X)), random_state=42)
preprocessor.fit(sample_fit)

# Aplicar por partes para manejar memoria
chunksize = 100000
X_transformed_list = []
for i in range(0, len(X), chunksize):
    end = i + chunksize
    chunk = X.iloc[i:end]
    chunk_trans = preprocessor.transform(chunk)
    X_transformed_list.append(chunk_trans)
    print(f"  → Transformado bloque {i} a {end}")

X_trans = np.vstack(X_transformed_list)

# Crear DataFrame final
col_names = (
    num_cols +
    list(preprocessor.named_transformers_["cat"]
         .named_steps["onehot"].get_feature_names_out(cat_ohe))
)
df_final = pd.DataFrame(X_trans, columns=col_names)
df_final[target_col] = train_df[target_col]
df_final[f"{target_col}_encoded_auto"] = y.reset_index(drop=True)

# ------------------------------------------------------------
# 7. Verificación final
# ------------------------------------------------------------
print("\n--- VERIFICACIÓN FINAL ---")
print(f"Dimensiones finales: {df_final.shape}")
print(f"Valores nulos totales: {df_final.isna().sum().sum()}")
print(f"Últimas columnas: {df_final.columns[-5:].tolist()}")

# ------------------------------------------------------------
# 8. Guardar resultados
# ------------------------------------------------------------
os.makedirs("output", exist_ok=True)
csv_path = "output/train_processed.csv"
pipe_path = "output/preprocessing_pipeline.joblib"

df_final.to_csv(csv_path, index=False)
joblib.dump(preprocessor, pipe_path)

print(f"\n✅ CSV guardado en: {csv_path}")
print(f"✅ Pipeline guardado en: {pipe_path}")
print("\n--- PREPROCESAMIENTO COMPLETADO CON ÉXITO ---")
